# ARIMA and SARIMA models

This notebook has the purpose to create 2 models: 

- ARIMA model
- SARIMA model

These models can be used to forecast any type of time series, the goal with this notebook is to create these forecast models for domestic hot water (DHW) and energy consumptions (energy to heat the water)

For this notebook it will be created an ARIMA and a SARIMA model for one set of data as an example, but these models can be reproduced for any time series


**1) Import of the packages and libraries**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import seaborn as sn
from statsmodels.tsa.stattools import acf

**2) Import of the data**

The first thing is to select the data that it will be used, the data referent to DHW and energy consumptions for residential apartments will be used

A random data was chosen for this example: from 26/01/2021 to 31/01/2021

The important aspect is the duration, 5 days

All the code was written considering this duration, if changed it will be necessary to altere the size of training, testing and forescating, these topics will be discussed later in this notebook

Down below the data must be chosen, in this example the data chosen is: "ap.25 energy consumption"

In [ ]:
#Energy Consumption
df25 = pd.read_csv ('./data_csv/25-IECS.csv', parse_dates=['Date'], index_col=['Date'])
df25 = df25 ['2021-01-26 00:00:00' : '2021-01-31 00:00:00']

**3) Resample the data in a time step of 1 hour**

The data is measured approximately each 3 minutes, so in order to have a better analysis during 5 days a resample of the data in a time step of 1 hour will be done

Resampling the data in time steps of 1 hour it will give 120 values

In [ ]:
#Energy consumption
s = df25.Value.resample('H').mean() 

**4) Visualization of the data**

In [ ]:
#Energy
plt.figure(figsize=(15,6))
plt.plot(s, label='ap.25', lw=3 ) 
plt.xlabel('Date')
plt.ylabel('Power (W)')
plt.title('Energy consumption', fontsize=15)
plt.legend(loc='upper left', fontsize=20)
plt.show()

**5) Verify if the data is stationary using Dickey-Fuller**

Normally if p-value is under 0,05 the data is considered as stationary. Otherwise differentiation will be necessary until the p-value is lower or equal to 0,05

To verify if the data is stationary a Dickey-Fuller test is used. The test will be done without differentiaition and with 1 differentiation for the data chosen

To verify with 2 differentiation, it is necessary to add "diff().diff()." after "s.", for 3,4... differentiaiton the same principle is applied


**5.1) Without differentiation**

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(s.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

**5.2) With 1 differentiation**

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
result1 = adfuller(s.diff().dropna())
print('ADF Statistic: %f' % result1[0])
print('p-value: %f' % result1[1])

In the example the data is already stationary without differentiation, so we can consider "d=0", otherwise the "d" equals the number of differentiations

**6) Division of the data in training and testing**

A division of the data is necessary, one part for train the model and a second part for test the model

It will be considered a ratio of 80:20 

- 80% training
- 20% testing

In this case (5 days with time step of 1h) there are 120 values, so 96 values are for training and 24 are for testing

In [ ]:
train = s[:96]   
test = s[96:]

**7) ARIMA Model**

ARIMA is a univariate time series model with 3 components: 

- Autoregressive (AR): responsible for forecasting future points from a linear regression
- Integrate Average (I): resposible for the differentiation of the data
- Moving Average (MA): responsible for performing a regression to forecast future values, but in contrast to the AR component, this regression is based on past errors and not on past observations.

ARIMA models are frequently indicated as ARIMA (p,d,q), where:

- p represents the AR component
- d represents the I component
- q represents the MA component



**8) Creation of the model using AUTO ARIMA**

AUTO ARIMA automatically selects the parameters of ARIMA model

The inputs are the data for trainig the model (in this case the variable "train") and d (differentiation parameter)

- d=0 data is stationary
- d>0 when data was differentiated

Even though Dickey-Fuller test gives the parameter "d", sometimes the model that was more suitible using another "d".

For this raison it is strongly recommended to test for d=0 and d=1 (usually 1 differentiation is enough) and compare the results to choose the model that is more suitible


For a better understanding of the function:
https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html

In [ ]:
#! pip install pmdarima
import pmdarima as pm
model_auto = pm.auto_arima(train, d=0, trace=False)                 
print(model_auto.summary())

The model summary gives the most suitible ARIMA model (p,d,q), in this case ARIMA (1,0,1)

in model_auto1 = ARIMA (train, order=(1, 0, 1)) replace the values for the parameters (p,d,q) chosen by AUTO ARIMA, in the example (1,0,1)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA  
model_auto1 = ARIMA(train, order=(1, 0, 1)) 
fitted_auto1 = model_auto1.fit() 
fc_auto = fitted_auto1.forecast(24, alpha=0.05)


fc_auto is the forescast, in this case it has 24 values (20% testing)

**8.1) Plot of the testing, traininig and forescast (ARIMA Model)**

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training') 
plt.plot(test, label='test') 
plt.plot(fc_auto, label='forecast ARIMA') 
plt.title('Forecast vs training and testing')
plt.xlabel('Date')
plt.ylabel('Power (W)')
plt.legend(loc='upper left', fontsize=8)
plt.show()

**9) Model evaluation**

3 types of evaluation will be done :

- Mean Absolute Percentage Error (MAPE): 

$$\frac{1}{n} \sum_{t=1}^{n}\frac{|F_{t} - A_{t}|} {|A_{t}|}$$

- Mean Absolute Error (MAE):

$$\frac{1}{n} \sum_{t=1}^{n}|F_{t} - A_{t}|$$

        Where the values are:

$$F_{t}=forecast$$
$$A_{t}=actual$$

- Correlation

In [ ]:
def forecast_accuracy_auto (forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))
    mae = np.mean(np.abs(forecast - actual))   
    corr = (np.corrcoef(forecast, actual)[0,1]) * 100   
    
    return({'mape':mape, 'mae': mae, 'correlation (%)':corr})

Error_Arima = forecast_accuracy_auto (fc_auto, test)
print(Error_Arima)

A comparison of the model using d=0 and d=1 must be done as mentionned, for changing the parameter "d" go back to the section 8

The selection of the model is based in 3 criteria:

- Analysis of the errors

- Correlation

- Visual analysis of the graphs

**In this case the model chosen is the ARIMA (1,0,1)**

**10) SARIMA model**

Seasonal ARIMA or SARIMA is the ARIMA model but including seasonality

SARIMA has 7 parameters, the same 3 from ARIMA with additional 4, which are:
- P (seasonal autoregressive terms)
- D (number of seasonal differences)
- Q (number of seasonal lagged forecast errors in the prediction equation)
- m (number of periods in each season)

SARIMA model are referred as SARIMA (p,d,q) x (P,D,Q) [m]

**11) Creation of the second model using AUTO SARIMA**

In [ ]:
#!pip3 install pyramid-arima
import pmdarima as pm


The same function used in item 8 will be used, but this time adding 4 new parameters mentionned before (P,D,Q)[m]

These parameters are releated with the seasonality of the data

The paramater "m" must be chosen by the user. The choice of the parameter is by analyzing the data, for example monthly data have m=12.

In the case of energy consumption m=24 showed to be suitible.

The same analysis made for ARIMA concerning the parameter "d" must be done for SARIMA for the same reasons

In [ ]:
smodel = pm.auto_arima(train,test='adf',max_p=3, max_q=3, m=24,start_P=0, seasonal=True,d=1, D=1, trace=False, 
                       error_action='ignore', suppress_warnings=True, stepwise=True)

smodel.summary()

The smodel summary gives the most suitible ARIMA model (p,d,q ) x (P,D,Q,m)

in smodel1 = ARIMA (train, order=(0, 1, 0), seasonal_order = (1,1,0,24)) replace the values for the parameters (p,d,q) (P,D,Q,m) chosen by AUTO ARIMA

In [ ]:
smodel1 = ARIMA(train, order=(0, 1, 0), seasonal_order = (1, 1, 0, 24)) 
fitted_smodel1 = smodel1.fit()
fc_smodel = fitted_smodel1.forecast(24, alpha=0.05) 

fc_smodel is the forescast, in this case it has 24 values (20% testing)

**11.1) Plot of the testing, traininig and forescast (SARIMA Model)**

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training') 
plt.plot(test, label='test') 
plt.plot(fc_smodel, label='forecast SARIMA') 
plt.title('Forecast vs training and testing')
plt.xlabel('Date')
plt.ylabel('Power (W)')
plt.legend(loc='upper left', fontsize=8)
plt.show()

**12) Model Evaluation**

In this section the same model evaluations from the section 9 were used

In [ ]:
def forecast_accuracy_SARIMA (forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))
    mae = np.mean(np.abs(forecast - actual))    
    corr = (np.corrcoef(forecast, actual)[0,1]) * 100   
   
    return({'mape':mape,'mae': mae, 'correlation(%)':corr})

Error_Sarima = forecast_accuracy_SARIMA (fc_smodel, test)
print(Error_Sarima)

As in the ARIMA model, when analyzing SARIMA model a comparison using d=0 and d=1 must be done, it is due the same reasons mentionned for the ARIMA model

**In this case the model chosen is SARIMA (0,1,0) x (1,1,0)[24]**

**13) Comparison between ARIMA and SARIMA**

Now a comparison of the 2 models is possible

In [ ]:
Error_Arima = pd.DataFrame(list(Error_Arima.items()),columns = ['Loss function','Value'])
Error_Sarima = pd.DataFrame(list(Error_Sarima.items()),columns = ['Loss function','Value'])
Comp_A_S = Error_Arima[['Loss function','Value']].copy()

Comp_A_S.insert(1, 'Value Sarima', Error_Sarima.Value)
Comp_A_S.rename(columns = {'Value':'Value ARIMA'}, inplace = True)

print(Comp_A_S)

In [ ]:
Comp_A_S.plot(kind = 'bar', x = 'Loss function')
plt.title('ARIMA X SARIMA', fontsize=15)
plt.legend(loc='upper right', fontsize=10)
plt.show()

In this example the SARIMA model showed a lower MAE and a higher correlation than the ARIMA model

It can be concluded that the SARIMA model was capable to predict with a high accuracy the pattern of the energy consumption for this apartment

For the creation of the models it was used the tutorial available at: https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/